## Edinburgh Living Landscape Pollinator Pledge

https://edinburghlivinglandscape.org.uk/pollinatorpledge/

In [41]:
import logging
import sys

import folium
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from shapely.geometry import Point

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
fn = '../data/interim/swt_pollinator_data-2019-02-15.csv'
pollinator_data = pd.read_csv(fn, parse_dates=['Entry Date'])
pollinator_data = pollinator_data.rename(columns={'What type of space do you have?': 'type'})

In [43]:
df = pollinator_data[['latitude', 'longitude', 'type']]
df = df.dropna()
#df = df.drop('Entry Date', axis=1)
df.head()

,latitude,longitude,type
0,55.938051,-3.217624,Communal greenspace
1,55.925941,-3.277307,Small garden
2,55.943268,-3.288348,Large garden
3,55.956412,-3.290882,Small garden
4,55.899452,-3.218028,Small garden


In [52]:
locations = zip(df.latitude, df.longitude, df.type)
locations = list(zip(df.latitude, df.longitude, df.type))
locations[:10]

[(55.93805129999999, -3.2176237000000003, 'Communal greenspace'),
 (55.9259405, -3.2773065000000003, 'Small garden'),
 (55.943267500000005, -3.2883483, 'Large garden'),
 (55.9564125, -3.290882, 'Small garden'),
 (55.899451899999995, -3.2180276, 'Small garden'),
 (55.9247822, -3.2689695, 'Small garden'),
 (55.864465100000004, -3.0619456, 'Small garden'),
 (55.9161095, -3.2825452000000004, 'Small garden'),
 (55.93220720000001, -3.2380139999999997, 'Small garden'),
 (55.9597619, -3.3996525, 'Small garden'),
 (55.9444115, -3.2209011000000003, 'Balcony / window box'),
 (55.960864799999996, -3.1632675999999997, 'Balcony / window box'),
 (55.92632629999999, -3.1877931, 'Large garden'),
 (55.913799, -3.288885, 'Communal greenspace'),
 (55.9378437, -3.2097109, 'Communal greenspace'),
 (55.939713299999994, -3.251081, 'Small garden'),
 (55.94788539999999, -3.2775295, 'Large garden'),
 (55.935447499999995, -3.1701732000000002, 'Large garden'),
 (55.9475635, -3.1425928, 'Large garden'),
 (55.946015

In [68]:
tiles = "openstreetmap"
edinburgh_centre = (55.953251, -3.188267)

m = folium.Map(location=edinburgh_centre, tiles=tiles, zoom_start=12)

for loc in locations:
    point = [loc[0], loc[1]]
    folium.CircleMarker(location=point,                      
                        radius = 5,
                        popup= loc[2],
                        color = 'red',
                        weight = 1,
                        fill='true',
                        fill_color='red',
                        fill_opacity=0.25).add_to(m)

m.save('pollinators.html')

m

In [32]:
# locations = list(zip(pollinator_df['latitude'], pollinator_df['longitude']))
# locations[:10]
# tiles = "openstreetmap"
# edinburgh_centre = (55.953251, -3.188267)

# m = folium.Map(location=edinburgh_centre, tiles=tiles, zoom_start=12)

# for location in locations:
#     folium.CircleMarker(location=location, 
#                         radius=5,
#                         stroke=1,
#                         color='darkblue',
#                         fill_color='darkblue').add_to(m)
# m

In [33]:
points = [Point(x, y) for x, y in zip(df.longitude, df.latitude)]
polli_gdf = gpd.GeoDataFrame(df, geometry=points)
polli_gdf.crs = {"init": "epsg:4326"}
polli_gdf.shape

(180, 4)

In [34]:
# tiles = "openstreetmap"
# edinburgh_centre = (55.953251, -3.188267)

# m = folium.Map(location=edinburgh_centre, tiles=tiles, zoom_start=12)

# style_function = lambda x: {"fillColor": "#00FFFFFF", "color": "#000000"}

# polli_geo = folium.GeoJson(
#     polli_gdf,
#     tooltip=folium.GeoJsonTooltip(
#         fields=["type"],
#         labels=True,
#         sticky=False,
#     ),
#     style_function=style_function
# )

# m.add_child(polli_geo)
# #m.save("images/greenspace.html")
# m